In [1]:
!mkdir dataset
!mkdir dataset/images
!mkdir dataset/labels
!mkdir dataset/images/train
!mkdir dataset/images/val
!mkdir dataset/labels/train
!mkdir dataset/labels/val
!mkdir dataset/images/test
!mkdir dataset/labels/test

In [2]:
import os
images=os.listdir('/kaggle/input/lidc-idri-preprocessed/CT images and masks/train/images')
print(len(images))

14343


In [3]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


In [4]:
bening_slices=0
malignant_slices=0
non_nodule_slices=0
bening_total=0
malignant_total=0

In [5]:
import numpy as np
from tqdm import tqdm
import shutil
from PIL import Image
import cv2
import os
import random


def annotations(mask) :
    ans=[]
    num_labels, labels = cv2.connectedComponents(mask.astype(np.uint8))
    #print(num_labels, labels)
    for i in range(1, num_labels) :
        ys, xs = np.where(labels == i)
        min_x, max_x = xs.min(), xs.max()
        min_y, max_y = ys.min(), ys.max()
        class_id=mask[ys[0], xs[0]]
        #print(class_id, xs[0],ys[0])
        x_center=(min_x+max_x)/2/512
        y_center=(min_y+max_y)/2/512
        width=(max_x-min_x+2)/512
        height=(max_y-min_y+2)/512
        ans.append([class_id, x_center, y_center, width, height])
    return ans

images=os.listdir('/kaggle/input/lidc-idri-preprocessed/CT images and masks/train/images')
for i  in tqdm(range(len(images))) :
    CASE_IDENTIFIER=str(i).zfill(5)
    


    
    old_path = '/kaggle/input/lidc-idri-preprocessed/CT images and masks/train/masks/'+images[i]
    img = Image.open(old_path)
    img = np.array(img, dtype=np.uint8)
    if np.max(img)==2 :
        for j in range(3) :
            if j==1 :
                img1=np.fliplr(img)
            elif j==2 :
                img1=np.flipud(img)
            else :
                img1=img
            anns=annotations(img1)
            new_path='/kaggle/working/dataset/labels/train/'+CASE_IDENTIFIER+'_'+str(j)+'.txt'
            bening_flag=0
            malignant_flag=0
            with open(new_path, "w", encoding="utf-8") as f:
                for ann in anns:
                    if ann[0]==2 :
                        f.write(f'{1} {ann[1]} {ann[2]} {ann[3]} {ann[4]}\n')
                        
                        malignant_flag=1
                        malignant_total+=1
                    else :
                        bening_total+=1
                        bening_flag=1
                if malignant_flag :
                    malignant_slices+=1
                if bening_flag :
                    bening_slices+=1
            old_path = '/kaggle/input/lidc-idri-preprocessed/CT images and masks/train/images/'+images[i]
            image = Image.open(old_path)
            image = np.array(image, dtype=np.uint8)
            if j==1 :
                img1=np.fliplr(image)
            elif j==2 :
                img1=np.flipud(image)
            else :
                img1=image
            new_path='/kaggle/working/dataset/images/train/'+CASE_IDENTIFIER+'_'+str(j)+'.png'
            img1 = Image.fromarray(img1)
            img1.save(new_path)




    else :
        if np.max(img)==1 :
            anns=annotations(img)
            new_path='/kaggle/working/dataset/labels/train/'+CASE_IDENTIFIER+'.txt'
            bening_flag=0
            with open(new_path, "w", encoding="utf-8") as f:
                for ann in anns:
                    bening_flag=1
                    bening_total+=1
                    if ann[0]!=1 :
                        f.write(f'{0} {ann[1]} {ann[2]} {ann[3]} {ann[4]}\n')
                if bening_flag :
                    bening_slices+=1
            old_path = '/kaggle/input/lidc-idri-preprocessed/CT images and masks/train/images/'+images[i]
            new_path='/kaggle/working/dataset/images/train/'+CASE_IDENTIFIER+'.png'
            shutil.copy(old_path, new_path)
        elif random.random()<0.2 :
            anns=annotations(img)
            new_path='/kaggle/working/dataset/labels/train/'+CASE_IDENTIFIER+'.txt'
            non_nodule_slices+=1
            with open(new_path, "w", encoding="utf-8") as f:
                for ann in anns:
                    if ann[0]!=1 :
                        f.write(f'{0} {ann[1]} {ann[2]} {ann[3]} {ann[4]}\n')
        
            old_path = '/kaggle/input/lidc-idri-preprocessed/CT images and masks/train/images/'+images[i]
            new_path='/kaggle/working/dataset/images/train/'+CASE_IDENTIFIER+'.png'
            shutil.copy(old_path, new_path)







100%|██████████| 14343/14343 [05:30<00:00, 43.34it/s]


In [6]:
len(os.listdir('/kaggle/working/dataset/images/train/'))

7878

In [7]:
print(bening_slices,
malignant_slices,
non_nodule_slices,
bening_total,
malignant_total)

897 4749 2328 936 4956


In [8]:
bening_slices=0
malignant_slices=0
non_nodule_slices=0
bening_total=0
malignant_total=0

In [9]:



images=os.listdir('/kaggle/input/lidc-idri-preprocessed/CT images and masks/val/images')
for i  in tqdm(range(len(images))) :
    CASE_IDENTIFIER=str(i).zfill(5)


    
    old_path = '/kaggle/input/lidc-idri-preprocessed/CT images and masks/val/masks/'+images[i]
    img = Image.open(old_path)
    img = np.array(img, dtype=np.uint8)
    bening_flag=0
    malignant_flag=0
    if np.max(img)<3 :
        anns=annotations(img)
        new_path='/kaggle/working/dataset/labels/val/'+CASE_IDENTIFIER+'.txt'
        with open(new_path, "w", encoding="utf-8") as f:
            for ann in anns:
                if ann[0]==2 :
                    malignant_flag=1
                    malignant_total+=1
                    f.write(f'{1} {ann[1]} {ann[2]} {ann[3]} {ann[4]}\n')
                else :
                    bening_flag=1
                    bening_total+=1

            if malignant_flag :
                malignant_slices+=1
            if bening_flag :
                bening_slices+=1
            if malignant_flag==0 and bening_flag==0 :
                non_nodule_slices+=1
        old_path = '/kaggle/input/lidc-idri-preprocessed/CT images and masks/val/images/'+images[i]
        new_path='/kaggle/working/dataset/images/val/'+CASE_IDENTIFIER+'.png'
        shutil.copy(old_path, new_path)

100%|██████████| 3391/3391 [00:57<00:00, 58.65it/s]


In [10]:
print(bening_slices,
malignant_slices,
non_nodule_slices,
bening_total,
malignant_total)

176 350 2659 177 373


In [12]:
bening_slices=0
malignant_slices=0
non_nodule_slices=0
bening_total=0
malignant_total=0

In [13]:

images=os.listdir('/kaggle/input/lidc-idri-preprocessed/CT images and masks/test/images')
for i  in tqdm(range(len(images))) :
    CASE_IDENTIFIER=str(i).zfill(5)


    
    old_path = '/kaggle/input/lidc-idri-preprocessed/CT images and masks/test/masks/'+images[i]
    img = Image.open(old_path)
    img = np.array(img, dtype=np.uint8)
    bening_flag=0
    malignant_flag=0
    if np.max(img)<3 :
        anns=annotations(img)
        new_path='/kaggle/working/dataset/labels/test/'+CASE_IDENTIFIER+'.txt'
        with open(new_path, "w", encoding="utf-8") as f:
            for ann in anns:
                if ann[0]==2 :
                    malignant_flag=1
                    malignant_total+=1
                    f.write(f'{1} {ann[1]} {ann[2]} {ann[3]} {ann[4]}\n')
                else :
                    bening_flag=1
                    bening_total+=1

            if malignant_flag :
                malignant_slices+=1
            if bening_flag :
                bening_slices+=1
            if malignant_flag==0 and bening_flag==0 :
                non_nodule_slices+=1
        
        old_path = '/kaggle/input/lidc-idri-preprocessed/CT images and masks/test/images/'+images[i]
        new_path='/kaggle/working/dataset/images/test/'+CASE_IDENTIFIER+'.png'
        shutil.copy(old_path, new_path)

100%|██████████| 3330/3330 [00:58<00:00, 56.83it/s]


In [14]:
print(bening_slices,
malignant_slices,
non_nodule_slices,
bening_total,
malignant_total)

148 276 2725 150 286
